이 프로젝트는 구글 코랩 환경에서 진행되었습니다

In [ ]:
from google.colab import drive   
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/
! git clone https://github.com/coqui-ai/TTS  #프로젝트에서 사용할 XTTS-v2 다운로드

/content/drive/MyDrive
Cloning into 'TTS'...
remote: Enumerating objects: 32844, done.
remote: Counting objects: 100% (2774/2774), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 32844 (delta 2725), reused 2701 (delta 2701), pack-reused 30070 (from 1)
Receiving objects: 100% (32844/32844), 166.14 MiB | 17.09 MiB/s, done.
Resolving deltas: 100% (23844/23844), done.
Updating files: 100% (725/725), done.


In [ ]:
%cd TTS #TTS 설치
%pip install -e .

/content/TTS
Obtaining file:///content/TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
Using cached numpy-1.22.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
  Building editable for TTS (pyproject.toml) ... done
  Created wheel for TTS: filename=TTS-0.22.0-0.editable-cp310-cp310-linux_x86_64.whl size=15043 sha256=e16eed1771e066242f15f3266cb4a7f8ef8cfb203cabde2718496839d3ec6c0f
  Stored in directory: /tmp/pip-ephem-wheel-cache-rxt9s5w2/wheels/c4/ee/51/7ae272f137a19c06a50ae47c926e3eb3862d04b1e0fb0372ba
Successfully built TTS
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.0
    Uninstalling numpy-1.23.0:
      Successfully uninstalled numpy-1.23.0
  Attempting uninstall: TTS

In [ ]:
%pip install Trainer #학습에 사용할 Trainer 설치

/content/drive/MyDrive/TTS


In [ ]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

# 매개변수 정의
RUN_NAME = "XTTS_v2.0_FT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None
OUT_PATH = "/content/drive/MyDrive/run/training"

In [ ]:
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True
START_WITH_EVAL = True
BATCH_SIZE = 3
GRAD_ACUMM_STEPS = 84

#데이터셋 구성
config_dataset = BaseDatasetConfig(
    formatter="kss", #한국어 음성 데이터셋을 포맷으로 설정(TTS/TTS/tts/datasets/formatters.py 파일을 열서 언어에 맞는 포맷으로 수정)
    dataset_name="hijeong",
    path="/content/drive/MyDrive/hijeong", #데이터셋 파일이 저장된 디렉터리 경로
    meta_file_train="/content/drive/MyDrive/hijeong/metadata.txt",#학습용 메타데이터파일 경로
    language="ko",
)
DATASETS_CONFIG_LIST = [config_dataset] #데이터셋 설정을 리스트 형태로 저장

In [ ]:
#체크포인트 저장 경로 설정
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)

#외부 체크포인트 파일을 링크하고 저장 경로 설정
DVAE_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/dvae.pth"
MEL_NORM_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/mel_stats.pth"
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))


In [ ]:
#DVAE_CHECKPOINT 파일 다운로드 및 저장
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


TOKENIZER_FILE_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/vocab.json"
XTTS_CHECKPOINT_LINK = "https://coqui.gateway.scarf.sh/hf-coqui/XTTS-v2/main/model.pth"
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json 형식
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth 형식

if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )

 > Downloading DVAE files!


  0%|          | 0.00/1.07k [00:00<?, ?iB/s]
100%|██████████| 1.07k/1.07k [00:00<00:00, 4.29kiB/s]

  2%|▏         | 3.73M/211M [00:00<00:05, 36.8MiB/s]
  4%|▍         | 7.93M/211M [00:00<00:05, 39.8MiB/s]
  6%|▌         | 12.3M/211M [00:00<00:04, 41.4MiB/s]
  8%|▊         | 16.6M/211M [00:00<00:04, 42.1MiB/s]
 10%|▉         | 21.0M/211M [00:00<00:04, 42.8MiB/s]
 12%|█▏        | 25.3M/211M [00:00<00:04, 42.2MiB/s]
 14%|█▍        | 29.6M/211M [00:00<00:04, 42.6MiB/s]
 16%|█▌        | 33.9M/211M [00:00<00:04, 42.3MiB/s]
 18%|█▊        | 38.3M/211M [00:00<00:04, 42.7MiB/s]
 20%|██        | 42.6M/211M [00:01<00:03, 42.9MiB/s]
 22%|██▏       | 46.9M/211M [00:01<00:03, 41.6MiB/s]
 24%|██▍       | 51.1M/211M [00:01<00:04, 38.8MiB/s]
 27%|██▋       | 55.9M/211M [00:01<00:03, 41.6MiB/s]
 29%|██▉       | 61.2M/211M [00:01<00:03, 44.7MiB/s]
 31%|███       | 65.7M/211M [00:01<00:03, 44.4MiB/s]
 33%|███▎      | 70.2M/211M [00:01<00:03, 44.1MiB/s]
 35%|███▌      | 74.6M/211M [00:01<00:03, 43.6MiB/s]

 > Downloading XTTS v2.0 files!


100%|██████████| 211M/211M [00:05<00:00, 40.8MiB/s]

100%|██████████| 361k/361k [00:00<00:00, 1.35MiB/s]

  0%|          | 4.13M/1.87G [00:00<00:45, 40.9MiB/s]
  0%|          | 8.43M/1.87G [00:00<00:44, 42.2MiB/s]
  1%|          | 12.8M/1.87G [00:00<00:43, 42.7MiB/s]
  1%|          | 17.1M/1.87G [00:00<00:43, 42.6MiB/s]
  1%|          | 21.4M/1.87G [00:00<00:43, 42.7MiB/s]
  1%|▏         | 25.7M/1.87G [00:00<00:43, 42.6MiB/s]
  2%|▏         | 30.0M/1.87G [00:00<00:43, 42.3MiB/s]
  2%|▏         | 34.3M/1.87G [00:00<00:43, 42.5MiB/s]
  2%|▏         | 38.6M/1.87G [00:00<00:42, 42.6MiB/s]
  2%|▏         | 42.9M/1.87G [00:01<00:42, 42.8MiB/s]
  3%|▎         | 47.3M/1.87G [00:01<00:42, 42.8MiB/s]
  3%|▎         | 51.5M/1.87G [00:01<00:42, 42.7MiB/s]
  3%|▎         | 55.8M/1.87G [00:01<00:42, 42.6MiB/s]
  3%|▎         | 60.2M/1.87G [00:01<00:42, 42.9MiB/s]
  3%|▎         | 64.5M/1.87G [00:01<00:42, 42.8MiB/s]
  4%|▎         | 68.7M/1.87G [00:01<00:42, 42.7MiB/s]
  4%|▍         | 73.0M/1.87G [

In [ ]:

SPEAKER_REFERENCE = [
    "/content/drive/MyDrive/hijeong/wavs/audio2.wav"
    
]
LANGUAGE = config_dataset.language

#모델 정의
def main():

    model_args = GPTArgs( 
        max_conditioning_length=132300,  
        min_conditioning_length=66150,  
        debug_loading_failures=False,
        max_wav_length=255995,
        max_text_length=200,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  
        tokenizer_file="/content/drive/MyDrive/run2/training/XTTS_v2.0_original_model_files/vocab.json",
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=24000)
    # 모델의 학습 설정
    config = GPTTrainerConfig(
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=50,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        print_eval=False,
        optimizer="AdamW",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  
        lr_scheduler="StepLR",

        lr_scheduler_params={"step_size": 50, "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
            {
                "text": "이것은 테스트 문장입니다",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
            {
                "text": "실제 목소리와 닮았나요?",
                "speaker_wav": SPEAKER_REFERENCE,
                "language": LANGUAGE,
            },
        ],
    )

    # 모델 초기화
    model = GPTTrainer.init_from_config(config)

    # 학습 샘플 로드
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # 트레이터 초기화
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()  #학습 실행


if __name__ == "__main__":
    main()


/content/drive/MyDrive/TTS/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)
/content/drive/MyDrive

>> DVAE weights restored from: /content/drive/MyDrive/run2/training/XTTS_v2.0_original_model_files/dvae.pth
 | > Found 408 files in /content/drive/MyDrive/hijeong


 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000

 > Model has 518442047 parameters

 > EPOCH: 0/1000
 --> /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000


 > Filtering invalid eval samples!!



 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.0515594482421875 (+0.002025604248046875)
     | > avg_loss_text_ce: 0.02862638235092163 (-0.00015665777027606964)
     | > avg_loss_mel_ce: 2.937391996383667 (-0.057863712310791016)
     | > avg_loss: 2.9660184383392334 (-0.05802035331726074)

 > BEST MODEL : /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000/best_model_1215.pth

 > EPOCH: 10/1000
 --> /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000

 > TRAINING (2024-12-04 12:46:32) 

   --> TIME: 2024-12-04 12:47:05 -- STEP: 35/135 -- GLOBAL_STEP: 1250
     | > loss_text_ce: 0.021962592378258705  (0.024776062635438783)
     | > loss_mel_ce: 2.7668237686157227  (2.7882324491228374)
     | > loss: 0.03319983929395676  (0.033488197305372794)
     | > current_lr: 5e-06 
     | > step_time: 0.4676  (0.6294276305607387)
     | > loader_time: 0.0322  (0.050618675776890366)


   --> TIME: 2024-12-04 12:47:50 -- STEP: 85/135 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.05261635780334473 (+0.0010569095611572266)
     | > avg_loss_text_ce: 0.02841506153345108 (-0.00021132081747055054)
     | > avg_loss_mel_ce: 2.897547721862793 (-0.03984427452087402)
     | > avg_loss: 2.9259626865386963 (-0.04005575180053711)

 > BEST MODEL : /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000/best_model_1350.pth

 > EPOCH: 11/1000
 --> /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000

 > TRAINING (2024-12-04 12:50:37) 

   --> TIME: 2024-12-04 12:50:43 -- STEP: 0/135 -- GLOBAL_STEP: 1350
     | > loss_text_ce: 0.023307839408516884  (0.023307839408516884)
     | > loss_mel_ce: 3.069133996963501  (3.069133996963501)
     | > loss: 0.03681478276848793  (0.03681478276848793)
     | > current_lr: 5e-06 
     | > step_time: 1.8012  (1.8012232780456543)
     | > loader_time: 3.392  (3.3920490741729736)


   --> TIME: 2024-12-04 12:51:29 -- STEP: 50/135 -- GLO

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.06128573417663574 (+0.008669376373291016)
     | > avg_loss_text_ce: 0.028217623010277748 (-0.00019743852317333221)
     | > avg_loss_mel_ce: 2.8707165718078613 (-0.02683115005493164)
     | > avg_loss: 2.8989341259002686 (-0.027028560638427734)

 > BEST MODEL : /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000/best_model_1485.pth

 > EPOCH: 12/1000
 --> /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000

 > TRAINING (2024-12-04 12:54:53) 

   --> TIME: 2024-12-04 12:55:17 -- STEP: 15/135 -- GLOBAL_STEP: 1500
     | > loss_text_ce: 0.023579690605401993  (0.02423814870417118)
     | > loss_mel_ce: 2.8191840648651123  (2.7642398675282798)
     | > loss: 0.03384242579340935  (0.03319616752366226)
     | > current_lr: 5e-06 
     | > step_time: 1.0037  (1.1085478941599527)
     | > loader_time: 0.1142  (0.10165945688883464)

 > Keyboard interrupt detected.
 > Saving model be

추후 학습을 이어서 따로 진행하였고 여러번의 테스트를 해본 결과 에폭이 많아도 유의미한 변화가 없다는 것을 알 수 있었습니다.
 (약 3시간 이후부터 큰 변화가 없었음)

In [ ]:
%load_ext tensorboard #텐서보드 로드
%tensorboard --logdir==/content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000

In [ ]:
#만든 모델로 텍스트 -> 음성 변환을 하기위해 best_model.pth를 model.pth로 이름 수정
!mv /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000/best_model.pth  /content/drive/MyDrive/run2/training/XTTS_v2.0_FT-December-04-2024_12+07PM-0000000/model.pth


In [ ]:
import os
import torch
import torchaudio
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts


print("Loading model...") #모델 로드
config = XttsConfig()
config.load_json("/content/drive/MyDrive/new/config.json")
model = Xtts.init_from_config(config)
model.load_checkpoint(config, checkpoint_dir="/content/drive/MyDrive/new/", use_deepspeed=False)
model.cuda()

gpt_cond_latent, speaker_embedding = model.get_conditioning_latents(audio_path=["/content/녹음 (2).wav"]) #모델이 음성 특성을 추출할 파일
print("Inference...")
out = model.inference(
    "누구나 쉽게 배우는 딥러닝 스타트를 배우는 임희정 입니다.",
    "ko",
    gpt_cond_latent,
    speaker_embedding,
    temperature=0.7,
)
torchaudio.save("/content/drive/MyDrive/output/final5.wav", torch.tensor(out["wav"]).unsqueeze(0), 24000)

Loading model...
Inference...


In [ ]:
from IPython.display import Audio #만들어진 음성파일 재생

Audio('/content/drive/MyDrive/output/final5.wav')
